In [1]:
%pwd

'c:\\medicalchat-bot-ai\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\medicalchat-bot-ai'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [6]:
extracted_data=load_pdf_file(data='Data/')

In [7]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [9]:
import os
os.chdir("../")

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'},  # Change to 'cuda' if using GPU
        encode_kwargs={'normalize_embeddings': False}
    )
    return embeddings

embeddings = download_hugging_face_embeddings()

C:\Users\91970\AppData\Local\Temp\ipykernel_18992\3845790682.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\91970\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [26]:
from dotenv import load_dotenv
load_dotenv()

False

In [38]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [28]:
import os
print(os.getenv("PINECONE_API_KEY"))  # Should print your API key or None


None


In [29]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
pc = Pinecone(api_key="pcsk_4Qawp2_Jv6S8QbnCB9AjzPpvmqkVT7LHYgjRYkPaLjryuDdqd243SzCuJQgjADbgyWM2uS")
index_name = "medibot"
pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "medibot",
    "metric": "cosine",
    "host": "medibot-zp0tlf6.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = "add token here" 
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "add token here"

# Replace with your real API key


In [32]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [33]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [34]:
docsearch

In [35]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [36]:
retrieved_docs = retriever.invoke("What is Acne?")

In [37]:
retrieved_docs

[Document(id='0f1073b2-ae23-4eb2-a573-f51c18bd9c29', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='23ae96ca-5fb4-4945-bcc3-2784f598bfd1', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [ ]:
from langchain_community.llms import HuggingFaceHub
import os

# Set your Hugging Face API key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "add token here"

# Load Mistral-7B-Instruct from Hugging Face
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",  # ✅ Corrected model ID
    model_kwargs={"temperature": 0.4, "max_tokens": 500}
)


In [49]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [51]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [57]:
response = rag_chain.invoke({"input": "my symptoms are pimples on face what are the causes"})
print(response["answer"])


c:\Users\91970\anaconda3\envs\medibot\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


System: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.

skin. A pimple forms when the damaged follicle weakens
and bursts open, releasing sebum, bacteria, and skin and
white blood cells into the surrounding tissues. Inflamed
pimples near the skin’s surface are called papules; when
deeper, they are called pustules. The most severe type of
acne consists of cysts (closed sacs) and nodules (hard
swellings). Scarring occurs when new skin cells are laid
down to replace damaged cells.
The most common sites of acne are the face, chest,

greater susceptibility to the disease.
• Hormonal changes. Acne can flare up before menstrua-
tion, during pregnancy, and menopause.
• Diet. No foods cause acne, but certain foods may cause
flare-ups.
• Drugs. Acne can be a side effect of drugs including
tranquilizers, antidepre